In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from sklearn.datasets import make_blobs
from matplotlib.colors import ListedColormap
from matplotlib import cm
import math
np.set_printoptions(precision=2)
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [ ]:
SEED = 1234

keras.utils.set_random_seed(SEED)
np.random.seed(SEED)

In [ ]:
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, centers=centers, cluster_std=std, random_state=30
)

In [ ]:
def hide_header_footer_toolbar(canvas):
    canvas.header_visible = False
    canvas.footer_visible = False
    canvas.toolbar_visible = False


def plt_mc_data(
    ax,
    X,
    y,
    classes,
    class_labels=None,
    cmap="tab10",
    legend=False,
    size=50,
    m="o",
    equal_xy=False,
):
    for i in range(classes):
        idx = np.where(y == i)
        col = len(idx[0]) * [i]
        label = class_labels[i] if class_labels else "c{}".format(i)
        cmap = plt.get_cmap(cmap)
        ax.scatter(
            X[idx, 0],
            X[idx, 1],
            c=col,
            marker=m,
            cmap=cmap,
            vmin=0,
            vmax=cmap.N,
            s=size,
            label=label,
        )
    if legend:
        ax.legend()
    if equal_xy:
        ax.axis("equal")


def plt_mc(X_train, y_train, classes, centers, std):
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    hide_header_footer_toolbar(fig.canvas)
    plt_mc_data(
        ax,
        X_train,
        y_train,
        classes,
        cmap="tab10",
        legend=True,
        size=50,
        equal_xy=False,
    )
    ax.set_title("Multiclass Data")
    ax.set_xlabel("x0")
    ax.set_ylabel("x1")
    plt.show()

In [ ]:
plt_mc(X_train, y_train, classes, centers, std=std)

In [ ]:
# show classes in data set
print(f"Unique classes : {np.unique(y_train)}")
# show how classes are represented
print(f"Class representation : {y_train[:10]}")
# show shapes of our dataset
print(f"Shape of X_train : {X_train.shape}, Shape of y_train : {y_train.shape}")

In [ ]:
model = Sequential(
    [Dense(2, activation="relu", name="L1"), Dense(4, activation="linear", name="L2")]
)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(0.01),
)
model.fit(X_train, y_train, epochs=200)

In [ ]:
def plt_cat_decision_boundary_mc(ax, X, fn_pred, class_labels=None, legend=False):
    X0_min, X0_max = np.min(X[:, 0]) - 0.5, np.max(X[:, 0]) + 0.5
    X1_min, X1_max = np.min(X[:, 1]) - 0.5, np.max(X[:, 1]) + 0.5
    h = max(X0_max - X0_min, X1_max - X1_min) / 100
    X0, X1 = np.meshgrid(np.arange(X0_min, X0_max, h), np.arange(X1_min, X1_max, h))
    points = np.c_[X0.ravel(), X1.ravel()]
    Z = fn_pred(points)
    Z = Z.reshape(X0.shape)
    ax.contour(X0, X1, Z, lw=1)


def plt_cat_mc(X, y, model, classes):
    model_predict = lambda x: np.argmax(model.predict(x), axis=1)
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    hide_header_footer_toolbar(fig.canvas)
    plt_mc_data(ax, X, y, classes, cmap="tab10", legend=True)
    plt_cat_decision_boundary_mc(ax, X, model_predict)
    ax.set_title("Model Decision Boundary")
    plt.xlabel(r"$x_0$")
    plt.ylabel(r"$x_1$")
    plt.show()

In [ ]:
plt_cat_mc(X_train, y_train, model, classes)

In [ ]:
# gather the trained parameters from the first layer
W1, b1 = model.get_layer("L1").get_weights()
print(W1, b1)

In [ ]:
def plt_prob_z(ax, fn, x0_rng=(-8, 8), x1_rng=(-5, 4)):
    """plots a decision boundary but include shading to indicate the probability
    and adds a conouter to show where z=0
    """
    x0_space, x1_space = np.linspace(*x0_rng, 40), np.linspace(*x1_rng, 40)
    x0, x1 = np.meshgrid(x0_space, x1_space)
    grid_points = np.c_[x0.ravel(), x1.ravel()]
    z = fn(grid_points)
    z = z.reshape(x0.shape)
    c = np.where(z == 0, 0, 1)
    ax.contour(x0, x1, c, linewidths=1)
    cmap = ListedColormap(plt.get_cmap("Blues")(np.linspace(0.0, 0.9, 256)))
    pcm = ax.pcolormesh(
        x0,
        x1,
        z,
        norm=cm.colors.Normalize(vmin=np.amin(z), vmax=np.amax(z)),
        cmap=cmap,
        shading="nearest",
        alpha=0.9,
    )
    ax.figure.colorbar(pcm, ax=ax)


def plt_layer_relu(X, Y, W, b, classes):
    nunits = W.shape[1]
    Y = Y.reshape(
        -1,
    )
    fig, ax = plt.subplots(1, nunits, figsize=(7, 3))
    hide_header_footer_toolbar(fig.canvas)

    for i in range(nunits):
        layer_fn = lambda x: np.maximum(0, np.dot(x, W[:, i]) + b[i])
        plt_prob_z(ax[i], layer_fn)
        plt_mc_data(ax[i], X, Y, classes, cmap="tab10", legend=True, size=50, m="o")
        ax[i].set_title(f"Unit {i}")
        ax[i].set_ylabel(r"$x_1$", size=10)
        ax[i].set_xlabel(r"$x_0$", size=10)
    fig.tight_layout()
    plt.show()


# plot the function of the first layer
plt.close("all")
plt_layer_relu(
    X_train,
    y_train.reshape(
        -1,
    ),
    W1,
    b1,
    classes,
)

In [ ]:
# gather the trained parameters from the output layer
W2, b2 = model.get_layer("L2").get_weights()
# create the 'new features', the training examples after L1 transformation
X_L2 = np.maximum(0, (X_train @ W1 + b1))

In [ ]:
def plt_output_layer_linear(X, y, W, b, classes, x0_rng=None, x1_rng=None):
    y = y.reshape(
        -1,
    )
    nunits = W.shape[1]
    # 1. Decide on a fixed number of columns (e.g., 2 or 3)
    ncols = 2
    # 2. Use math.ceil to ensure we have enough rows for all units
    nrows = math.ceil(nunits / ncols)

    # 3. Create the subplots
    fig, ax = plt.subplots(nrows, ncols, figsize=(ncols * 3.5, nrows * 2.5))
    # 4. Flatten the axes for easy iteration
    # Note: if nrows*ncols > nunits, we'll have empty plots.
    # We turn them off later.
    axes = ax.flatten() if nunits > 1 else [ax]
    hide_header_footer_toolbar(fig.canvas)
    for i, axi in enumerate(ax.flat):
        layer_fn = lambda x: x @ W[:, i] + b[i]
        plt_prob_z(axi, layer_fn, x0_rng=x0_rng, x1_rng=x1_rng)
        plt_mc_data(axi, X, y, classes, cmap="tab10", legend=True, size=50, m="o")
        axi.set_xlabel(r"$a^{[1]}_0$", size=9)
        axi.set_ylabel(r"$a^{[1]}_1$", size=9)
        axi.set_xlim(x0_rng)
        axi.set_ylim(x1_rng)
        axi.set_title(f"Linear output unit {i}")
    fig.tight_layout()
    plt.show()


plt_output_layer_linear(
    X_L2,
    y_train,
    W2,
    b2,
    classes,
    x0_rng=(-0.25, np.max(X_L2[:, 0])),
    x1_rng=(-0.25, np.max(X_L2[:, 1])),
)